# Lab 3 — clustering

In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
import pickle
import sklearn.cluster as cl

# Import bokeh
from bokeh.io import show, output_notebook
from bokeh.plotting import figure, ColumnDataSource
from bokeh.models import HoverTool, ResetTool, PanTool, WheelZoomTool, SaveTool
output_notebook()
# Categorial colors
from bokeh.palettes import Dark2_8

%matplotlib inline
plt.style.use("ggplot")

Loading BokehJS ...

In [2]:
with open("arrrrr.pickle", "rb") as file:
    tagsPCA_p = pickle.load(file, encoding="utf-8")

In [3]:
tagsPCA = list(tagsPCA_p.values())
ks = [2, 3, 4, 5]

In [4]:
def visualize_cluster(k, data, names, dims):
    kmeans = cl.KMeans(k)
    kmeans.fit(data)
    print(kmeans)
    source = ColumnDataSource(
        data={
            "x": [x[dims[0]] for x in tagsPCA],
            "y": [x[dims[1]] for x in tagsPCA],
            "name": [x for x in names],
            "color": [Dark2_8[x] for x in kmeans.labels_],
        })

    hover = HoverTool(
        tooltips=[
            ("Name", "@name"),
        ])
    tools = [hover, ResetTool(), PanTool(), WheelZoomTool(), SaveTool()]
    p = figure(plot_width=960, plot_height=360, tools=tools, title="Mouse over the dots")
    p.circle("x", "y", source=source, size=20, color="color", alpha=0.5)
    show(p, notebook_handle=True)

In [5]:
for k in ks:
    print(k)
    visualize_cluster(k, tagsPCA, tagsPCA_p.keys(), (0, 1))

2
KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=2, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)


3
KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=3, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)


4
KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=4, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)


5
KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=5, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)


## 3.2

In [7]:
def J_distance(a, b):
    set1 = set(a)
    set2 = set(b)
    intersection = set1.intersection(set2)
    union = set1.union(set2)
    return 1 - ( intersection / union )

In [9]:
def k_mediods(points, dfun, k, threshold):
    
    #initialize mediods
    rand_index = np.random.randint(len(points), size=k)
    mediods = points[rand_index]
    mediods2 = np.zeros(medioids.shape)
    
    def converged():
        diff = mediods2-mediods
        max_dist = np.max(np.linalg.norm(diff, axis=0))
        return max_dist <= threshold
    
    #store distance of each cluster for each vector 
    dists = np.zeros((len(points), k))
    
    for i in range(len(points)):
        for j in range(k):
            dists[i, j] = J_distance(points[i], mediods[k])
    
    clusters = np.argpartition(dists,1,1)[:,0]
    while(not converged()):
        
        for x in points:
            for i in range(k):
                
        

SyntaxError: invalid syntax (<ipython-input-9-68dade497ded>, line 21)